#  Data Assembly Notebook - Bike Sharing Demand for NYC Citi Bike

A work of ***dataslays*** - your trustful partner for data science projects

<div class="alert alert-block alert-info">

This notebook provides the code and detailed instructions for acquiring and preparing the raw data required for the NYC Citi Bike demand analysis project.

</div>

# 0. Setup

In [1]:
import pandas as pd
import requests
import glob
import os
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

# 1. Citi Bike Trip Historical Data

<div class="alert alert-block alert-info">

The Citi Bike trip data is not available as a single file. Instead, it is distributed across multiple CSV files.

They can be downloaded directly from https://s3.amazonaws.com/tripdata/index.html.

Additional information about the data can be found on this page https://citibikenyc.com/system-data.

After downloading, all files are stored in the raw_data folder.

</div>

To analyze bike rental demand, I aggregate all data from 2024 to compute the number of classic bikes rented per hour each day.

In [2]:
csv_files = glob.glob('raw_data/*.csv')

In [3]:
all_raw_data = []
for file_path in csv_files:
    print(f"Loading: {os.path.basename(file_path)}")
    raw_data = pd.read_csv(file_path, low_memory=False)
    raw_data = raw_data[raw_data['rideable_type'] == 'classic_bike']
    all_raw_data.append(raw_data)

Loading: 202401-citibike-tripdata.csv
Loading: 202402-citibike-tripdata.csv
Loading: 202403-citibike-tripdata.csv
Loading: 202404-citibike-tripdata.csv
Loading: 202405-citibike-tripdata_1.csv
Loading: 202405-citibike-tripdata_2.csv
Loading: 202405-citibike-tripdata_3.csv
Loading: 202405-citibike-tripdata_4.csv
Loading: 202405-citibike-tripdata_5.csv
Loading: 202406-citibike-tripdata_1.csv
Loading: 202406-citibike-tripdata_2.csv
Loading: 202406-citibike-tripdata_3.csv
Loading: 202406-citibike-tripdata_4.csv
Loading: 202406-citibike-tripdata_5.csv
Loading: 202407-citibike-tripdata_1.csv
Loading: 202407-citibike-tripdata_2.csv
Loading: 202407-citibike-tripdata_3.csv
Loading: 202407-citibike-tripdata_4.csv
Loading: 202407-citibike-tripdata_5.csv
Loading: 202408-citibike-tripdata_1.csv
Loading: 202408-citibike-tripdata_2.csv
Loading: 202408-citibike-tripdata_3.csv
Loading: 202408-citibike-tripdata_4.csv
Loading: 202408-citibike-tripdata_5.csv
Loading: 202409-citibike-tripdata_1.csv
Loading:

After loading all relevant CSV files, I combine them into a single DataFrame for further processing.

In [4]:
raw_df = pd.concat(all_raw_data, ignore_index=True)
print(f"Total rows after combining: {len(raw_df):,}")

Total rows after combining: 15,056,634


To ensure data quality, I check for missing or duplicate ride IDs and remove any duplicates found.

In [5]:
raw_df['ride_id'].isna().sum()

0

In [6]:
raw_df['ride_id'].duplicated().sum()

382

In [7]:
raw_df[raw_df['ride_id'].duplicated(keep=False)].sort_values('ride_id')

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Unnamed: 0,rideable_type_duplicate_column_name_1
5197429,01282F20056F01F3,classic_bike,2024-05-31 21:04:43.872,2024-06-01 00:05:52.285,3 St & 7 Ave,3905.15,Grand Army Plaza & Plaza St West,4010.15,40.670384,-73.978397,40.672968,-73.970880,casual,NaN,NaN
3629991,01282F20056F01F3,classic_bike,2024-05-31 21:04:43,2024-06-01 00:05:52,3 St & 7 Ave,3905.15,Grand Army Plaza & Plaza St West,4010.15,40.670384,-73.978397,40.672968,-73.970880,casual,NaN,NaN
5802508,012DF189CC0CF68D,classic_bike,2024-05-31 23:47:45.292,2024-06-01 00:04:04.091,W 13 St & 7 Ave,6030.04,E 5 St & Ave C,5545.04,40.737815,-73.999947,40.722992,-73.979955,member,NaN,NaN
4820145,012DF189CC0CF68D,classic_bike,2024-05-31 23:47:45,2024-06-01 00:04:04,W 13 St & 7 Ave,6030.04,E 5 St & Ave C,5545.04,40.737815,-73.999947,40.722992,-73.979955,member,NaN,NaN
5509936,02A0DED3E1AC79A4,classic_bike,2024-05-31 23:57:25.546,2024-06-01 00:03:44.063,W 35 St & 8 Ave,6526.01,8 Ave & W 49 St,6747.07,40.752762,-73.992805,40.761712,-73.986551,member,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5141753,FE3B67C667F25BE7,classic_bike,2024-05-31 23:46:40.670,2024-06-01 00:20:50.938,E 32 St & Park Ave,6280.12,Metropolitan Ave & Bedford Ave,5308.04,40.745712,-73.981948,40.715348,-73.960241,member,NaN,NaN
5272729,FEFB55716D7687F7,classic_bike,2024-05-31 23:35:27.046,2024-06-02 00:35:19.593,Centre St & Worth St,5279.03,NaN,NaN,40.714948,-74.002345,NaN,NaN,casual,NaN,NaN
3521387,FEFB55716D7687F7,classic_bike,2024-05-31 23:35:27,2024-06-02 00:35:19,Centre St & Worth St,5279.03,NaN,NaN,40.714948,-74.002345,NaN,NaN,casual,NaN,NaN
3604025,FF0E6AED5C7686E6,classic_bike,2024-05-31 23:50:36,2024-06-01 00:00:49,W 27 St & 6 Ave,6215.07,8 Ave & W 31 St,6450.05,40.745260,-73.990620,40.750585,-73.994685,casual,NaN,NaN


In [8]:
raw_df = raw_df.drop_duplicates(subset=['ride_id'])
print(f"Rows after removing duplicates: {len(raw_df):,}")

Rows after removing duplicates: 15,056,252


There are different datetime formats in the data. Some entries have microseconds (%Y-%m-%d %H:%M:%S.%f) and others don't (%Y-%m-%d %H:%M:%S). I use format='mixed' parameter to correctly parse all date and time entries, then extract the date and hour for aggregation.

In [9]:
raw_df['started_at'] = pd.to_datetime(raw_df['started_at'], format='mixed', errors='coerce')
raw_df['date'] = raw_df['started_at'].dt.date
raw_df['hour'] = raw_df['started_at'].dt.hour

Next, I filter the data to make sure we include only trips from the year 2024. There are indeed some entries from 2023.

In [10]:
raw_df = raw_df[raw_df['started_at'].dt.year == 2024]
print(f"Rows after filtering for 2024: {len(raw_df):,}")

Rows after filtering for 2024: 15,056,124


I then aggregate the data to compute the hourly count of classic bike rentals for each day.

In [11]:
count_df = raw_df.groupby(['date', 'hour']).size().reset_index(name='rented_bike_count')
print(f"Final hourly aggregation has {len(count_df):,} rows")

Final hourly aggregation has 8,783 rows


Since 2024 is a leap year, I expect 366 days × 24 hours = 8,784 hourly entries. Since there are only 8783 entries in the dataframe, it's very likely that one entry is missing.

In [12]:
all_dates = pd.date_range(start=count_df['date'].min(), end=count_df['date'].max(), freq='D')
all_hours = range(24)

complete_combinations = []
for date in all_dates:
    for hour in all_hours:
        complete_combinations.append((date.date(), hour))

existing_combinations = set(zip(count_df['date'], count_df['hour']))
missing_combinations = set(complete_combinations) - existing_combinations

print(f"Missing date-hour combinations: {len(missing_combinations)}")
if len(missing_combinations) > 0:
    for combo in list(missing_combinations)[:10]:
        print(f"Date: {combo[0]}, Hour: {combo[1]}")

Missing date-hour combinations: 1
Date: 2024-03-10, Hour: 2


For plausibility, since the rented bike count at 2:00 AM is quite low, it is entirely possible that on March 10, 2024, at 2:00 AM, there were genuinely no classic bike rentals recorded.

In [13]:
count_df[count_df['hour'] == 2]['rented_bike_count'].nsmallest()

2089     7
1034    13
2233    15
386     16
362     18
Name: rented_bike_count, dtype: int64

For simplicity, I add this entry to the dataframe with count = 0

In [14]:
missing_row = {'date': '2024-03-10', 'hour': 2, 'rented_bike_count': 0}
count_df = pd.concat([count_df, pd.DataFrame([missing_row])], ignore_index=True).sort_values(['date', 'hour']).reset_index(drop=True)

In [15]:
count_df

,date,hour,rented_bike_count
0,2024-01-01,0,432
1,2024-01-01,1,583
2,2024-01-01,2,519
3,2024-01-01,3,272
4,2024-01-01,4,142
...,...,...,...
8779,2024-12-31,20,442
8780,2024-12-31,21,191
8781,2024-12-31,22,88
8782,2024-12-31,23,213


In [16]:
count_df['date'] = pd.to_datetime(count_df['date'])

# 2. Weather Data

<div class="alert alert-block alert-info">

Weather data is acquired using the open-source API from https://open-meteo.com/en/docs/historical-weather-api. 

Be sure to use the historical data endpoint, not the forecast endpoint.

</div>

I define a function to fetch hourly weather data for New York City between specified dates. The data includes temperature, humidity, wind speed, and several other relevant variables.

In [17]:
def get_weather_data(start_date, end_date):

    latitude = 40.7128
    longitude = -74.0060

    url = "https://archive-api.open-meteo.com/v1/archive"

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": [
            "temperature_2m",
            "relative_humidity_2m",
            "wind_speed_10m",
            "dew_point_2m",
            "shortwave_radiation",
            "rain",
            "snowfall",
            "cloud_cover",
            "apparent_temperature",
            "wind_gusts_10m",
            "sunshine_duration",
        ],
        "timezone": "America/New_York",
        "windspeed_unit": "ms",
    }

    response = requests.get(url, params=params)
    data = response.json()

    weather_df = pd.DataFrame(
        {
            "datetime": pd.to_datetime(data["hourly"]["time"]),
            "temperature": data["hourly"]["temperature_2m"],
            "humidity": data["hourly"]["relative_humidity_2m"],
            "wind_speed": data["hourly"]["wind_speed_10m"],
            "dew_point": data["hourly"]["dew_point_2m"],
            "solar_radiation": data["hourly"]["shortwave_radiation"],
            "rainfall": data["hourly"]["rain"],
            "snowfall": data["hourly"]["snowfall"],
            "cloud_cover": data["hourly"]["cloud_cover"],
            "apparent_temperature": data["hourly"]["apparent_temperature"],
            "wind_gusts": data["hourly"]["wind_gusts_10m"],
            "sunshine_duration": data["hourly"]["sunshine_duration"],
        }
    )

    weather_df["date"] = weather_df["datetime"].dt.date
    weather_df["hour"] = weather_df["datetime"].dt.hour

    weather_df["date"] = pd.to_datetime(weather_df["date"])

    return weather_df[
        [
            "date",
            "hour",
            "temperature",
            "humidity",
            "wind_speed",
            "dew_point",
            "solar_radiation",
            "rainfall",
            "snowfall",
            "cloud_cover",
            "apparent_temperature",
            "wind_gusts",
            "sunshine_duration",
        ]
    ]

For this proof of concept, I retrieve weather data for the entire year of 2024.

In [18]:
weather_data = get_weather_data("2024-01-01", "2024-12-31")

In [19]:
weather_data

,date,hour,temperature,humidity,wind_speed,dew_point,solar_radiation,rainfall,snowfall,cloud_cover,apparent_temperature,wind_gusts,sunshine_duration
0,2024-01-01,0,1.7,74,1.75,-2.4,0.0,0.0,0.0,100,-1.6,3.6,0.0
1,2024-01-01,1,1.7,76,2.46,-2.2,0.0,0.0,0.0,100,-2.0,3.8,0.0
2,2024-01-01,2,2.7,74,3.16,-1.5,0.0,0.0,0.0,100,-1.3,5.1,0.0
3,2024-01-01,3,2.8,74,2.69,-1.3,0.0,0.0,0.0,100,-0.9,4.9,0.0
4,2024-01-01,4,2.6,76,2.25,-1.2,0.0,0.0,0.0,65,-0.8,4.2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,2024-12-31,19,7.7,82,3.03,4.8,0.0,0.0,0.0,100,4.9,6.3,0.0
8780,2024-12-31,20,8.1,81,2.96,5.1,0.0,0.0,0.0,98,5.3,7.2,0.0
8781,2024-12-31,21,8.0,82,3.01,5.0,0.0,0.0,0.0,100,5.2,6.4,0.0
8782,2024-12-31,22,7.3,87,2.69,5.4,0.0,3.1,0.0,100,4.8,6.5,0.0


I then merge the hourly weather data with the hourly bike rental counts to create a comprehensive dataset.

In [20]:
final_df = count_df.merge(weather_data, on=['date', 'hour'], how='left')
final_df

,date,hour,rented_bike_count,temperature,humidity,wind_speed,dew_point,solar_radiation,rainfall,snowfall,cloud_cover,apparent_temperature,wind_gusts,sunshine_duration
0,2024-01-01,0,432,1.7,74,1.75,-2.4,0.0,0.0,0.0,100,-1.6,3.6,0.0
1,2024-01-01,1,583,1.7,76,2.46,-2.2,0.0,0.0,0.0,100,-2.0,3.8,0.0
2,2024-01-01,2,519,2.7,74,3.16,-1.5,0.0,0.0,0.0,100,-1.3,5.1,0.0
3,2024-01-01,3,272,2.8,74,2.69,-1.3,0.0,0.0,0.0,100,-0.9,4.9,0.0
4,2024-01-01,4,142,2.6,76,2.25,-1.2,0.0,0.0,0.0,65,-0.8,4.2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,2024-12-31,20,442,8.1,81,2.96,5.1,0.0,0.0,0.0,98,5.3,7.2,0.0
8780,2024-12-31,21,191,8.0,82,3.01,5.0,0.0,0.0,0.0,100,5.2,6.4,0.0
8781,2024-12-31,22,88,7.3,87,2.69,5.4,0.0,3.1,0.0,100,4.8,6.5,0.0
8782,2024-12-31,23,213,7.3,91,3.39,5.9,0.0,2.8,0.0,100,4.5,7.7,0.0


Finally, I store the assembled dataset for further analysis and modeling.

In [21]:
final_df.to_csv('data/raw_data.csv', index=False)